In [2]:
# We import the API key 
import config, os, openai, tiktoken, json
import numpy as np
os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY
os.environ['ORGANIZATION_ID'] = config.ORG_ID

In [3]:
# We check the dataset
with open('dataset.txt', encoding='latin-1') as f:
    text = [line for line in f]

In [4]:
# We see how difference each example in the dataset
text[:10]

['user: What type of house did you grow up in? \n',
 'assistant: I grew up in a farmhouse in the countryside, which had three bedrooms and a wood-burning sauna, which was built in the garage.\n',
 '-\n',
 'user: How would you describe your hometown? \n',
 'assistant: My hometown was a lovely, quiet town, but the industrial area had an odor due to the chemical plants.\n',
 '-\n',
 'user: Did you move a lot as a child or stay in one place? \n',
 'assistant: As a child, I lived on the same farm where my parents built a new house near an old house in the late 1970s.\n',
 '-\n',
 'user: What was your favorite room in your childhood house? \n']

We need to ensure that we follow the next format, according to Open IA documentation
```
{
  "messages": [
    { "role": "system", "content": "You are an assistant that occasionally misspells words" },
    { "role": "user", "content": "Tell me a story." },
    { "role": "assistant", "content": "One day a student went to schoool." }
  ]
}
```

In [5]:
# Function to refactor each example as the properly format
def refactor_example(example_list, system_message=None):
    messages = []

    # Include the general system message
    if system_message:
        messages.append({
            "role": "system",
            "content": system_message
        })

    # Iterate for each example
    for message in example_list:
        example = message.split(': ', maxsplit=1)

        #Control if each example dont follow the format
        if len(example) < 2:
            continue

        role = example[0].strip()
        content = example[1].strip()

        message_format = {
            "role": role,
            "content": content
        }

        messages.append(message_format)

    dict_final = {
        "messages": messages
    }

    return dict_final

In [6]:
# We apply the function in each example
system_message = 'You are a U.S.-born English teacher who spent 30 years in Finland. \
that likes philosophy and also likes to have conversations about his life, hobbies or teaching experiences.'

dataset = []

example = []
for line in text:
  if line == '-\n':
    format_example = refactor_example(example_list=example,
                                            system_message=system_message)

    dataset.append(format_example)
    example = []
    continue

  example.append(line)

In [7]:
dataset[1]

{'messages': [{'role': 'system',
   'content': 'You are a U.S.-born English teacher who spent 30 years in Finland. that likes philosophy and also likes to have conversations about his life, hobbies or teaching experiences.'},
  {'role': 'user', 'content': 'How would you describe your hometown?'},
  {'role': 'assistant',
   'content': 'My hometown was a lovely, quiet town, but the industrial area had an odor due to the chemical plants.'}]}

We check for errors and estimate the price using the guide [provided by OpenAI](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset)

In [8]:
from collections import defaultdict
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
            print(message)

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1
            print(message)

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1
            print(message)

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1
            print(message)

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1
        print(message)

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


We estimate the number of tokens and a aproximate of the cost of this fine tunning, based on [OpenIA GitHub](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)

In [9]:
encoding = tiktoken.get_encoding("cl100k_base")
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"average / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [10]:
# Warnings and tokens counts by each type of user
n_messages, convo_lens, assistant_message_lens = [], [], []

for ex in dataset:
    messages = ex["messages"]
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print_distribution(n_messages, "Number of messages for example")
print_distribution(convo_lens, "Number of token for example")
print_distribution(assistant_message_lens, "Number of tokens by assistant for example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples that exceed the token limit of 4096, they will be truncated during fine-tuning.")


#### Distribution of Number of messages for example:
min / max: 3, 5
average / median: 3.0129032258064514, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of Number of token for example:
min / max: 65, 177
average / median: 93.96774193548387, 90.0
p5 / p95: 73.4, 120.19999999999999

#### Distribution of Number of tokens by assistant for example:
min / max: 9, 111
average / median: 33.13548387096774, 29.0
p5 / p95: 14.0, 59.599999999999994

0 examples that exceed the token limit of 4096, they will be truncated during fine-tuning.


In [19]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 10
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"The dataset have ~{n_billing_tokens_in_dataset} tokens to use in the training phase")
print(f"By default, we will use {n_epochs} epochs in this training")
print(f"So, will be ~{n_epochs * n_billing_tokens_in_dataset} tokens in total")

The dataset have ~14565 tokens to use in the training phase
By default, we will use 10 epochs in this training
So, will be ~145650 tokens in total


Taking into account this official pricing in [Open IA Fine Tunnig - Fees](https://openai.com/pricing)

| Model         | Training            | Input usage         | Output usage        |   
|---------------|---------------------|---------------------|---------------------|
| GPT-3.5 Turbo | $0.0080 / 1K tokens | $0.0120 / 1K tokens | $0.0160 / 1K tokens |

In [20]:
training_cost = '${:,.2f}'.format(((n_epochs * n_billing_tokens_in_dataset)/1000)*0.008)
print(f'The estimate cost of the training is ~{training_cost}')

The estimate cost of the training is ~$1.17


In [13]:
# We save the dataset in the properly format (json)
with open('english_teacher_train_full.jsonl', 'w') as file:
    for ejemplo in dataset:
        json_line = json.dumps(ejemplo)
        file.write(json_line + '\n')

Fine Tunning Job in Open IA

In [14]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = "org-t4Sz2mkALC3ioqxKHOnyiQU1"

In [15]:
# We save the dataset and we save the id
train_full_response_file = openai.File.create(
    file=open('english_teacher_train_full.jsonl','rb'),
    purpose='fine-tune'
)


print(f'id: {train_full_response_file.id}')

id: file-ItmgJKWHlgzfWOArjGcMzKIM


In [16]:
# Create the job of the fine tunning
response = openai.FineTuningJob.create(training_file=train_full_response_file.id,
                                       model="gpt-3.5-turbo",
                                       suffix='english_teacher',
                                       hyperparameters={'n_epochs':4})

In [21]:
id_job = response.id
id_job

'ftjob-vZ4sWH6fVJ7bIsA3PsQUglbB'

In [22]:
response = openai.FineTuningJob.list_events(id=response.id)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])


Created fine-tuning job: ftjob-vZ4sWH6fVJ7bIsA3PsQUglbB
Validating training file: file-ItmgJKWHlgzfWOArjGcMzKIM
Files validated, moving job to queued state
Fine-tuning job started
Step 1/620: training loss=4.98
Step 11/620: training loss=3.81
Step 21/620: training loss=4.29
Step 31/620: training loss=2.05
